https://linqs.soe.ucsc.edu/data

In [1]:
from utils import load_data, split, accuracy
from model import GCN
import torch.optim as optim
import time
import torch.nn.functional as F
import numpy as np

import pandas as pd
import os
import pickle as pkl
import networkx as nx
import scipy.sparse as sp

### Feature

In [2]:
node_data = pd.read_csv(os.path.join("cora.content"), sep='\t', header=None)

In [3]:
node_data

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1128975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2704,1128977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2705,1128978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2706,117328,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based


In [4]:
feat = node_data.iloc[:,1:-1].astype(float)
feat = sp.csr_matrix(feat.values)

In [6]:
# with open('cora.feature', 'wb') as f:
#     pkl.dump(feat, f)

### Labels

In [7]:
labels = node_data.iloc[:,-1]
labels = labels.values

In [9]:
# with open('cora.labels', 'wb') as f:
#     pkl.dump(labels, f)

### Adjacency matrix

In [11]:
edgelist = pd.read_csv(os.path.join("cora.cites"), sep='\t', header=None, names=['source', 'target']); edgelist

,source,target
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


In [12]:
id2ind = dict(zip(node_data.iloc[:,0], node_data.index))

In [14]:
for i in edgelist.index:
    edgelist.iloc[i][0] = id2ind[edgelist.iloc[i][0]]
    edgelist.iloc[i][1] = id2ind[edgelist.iloc[i][1]]

In [15]:
edgelist

,source,target
0,163,402
1,163,659
2,163,1696
3,163,2295
4,163,1274
...,...,...
5424,1886,745
5425,1886,1902
5426,1887,2258
5427,1902,1887


In [16]:
G = nx.from_pandas_edgelist(edgelist)
adj_np = nx.to_numpy_matrix(G, nodelist=sorted(G.nodes()))
adj = sp.csr_matrix(adj_np)

In [18]:
# with open('cora.graph', 'wb') as f:
#     pkl.dump(adj, f)